In [ ]:
# Import required libraries
import math
import random
import numpy as np

# Import timings
import time
import os
import psutil

# Importing Pandas to create DataFrame
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt

from collections import Counter
from collections import defaultdict
from scipy.stats import gaussian_kde

#import seaborn as sns

# Create empty DataFrame with specific column names & types
df = pd.DataFrame({'Best weight': pd.Series(dtype='int'),
                   'Duration': pd.Series(dtype='float'),
                   'Iterations': pd.Series(dtype='int')})
df.index.rename('Experiment', inplace=True)

df["Best weight"] = df["Best weight"].astype(int)

df["Iterations"] = df["Iterations"].astype(int)



In [ ]:
# Create a highwaylist function that adds the differential characteristics to corresponding lists
def highwaylist():
    global stratum
    # Create 4 lists to hold the differential characteristics of each file
    alph,bet,gam,pro = [],[],[],[]
    with open('alph.txt') as ip1, open('bet.txt') as ip2, open('gam.txt') as ip3, open('pro.txt') as ip4:
        for i1,i2,i3,i4 in zip(ip1,ip2,ip3,ip4):
            # Append each differential to theappropriate list and remove whitespaces
            hw.append(NMTS(int(i1.strip()),int(i2.strip()),int(i3.strip()),float(i4.strip()))) 
            

            

In [ ]:
def sample_data(hw):
    # Set the sample size as a decimal percentage
    samepl_size = 0.05
    
    # group differentials by the dz attribute values
    grouped_data = defaultdict(list)
    
    # Loop through the hw list of dz differentials (the output diff)
    for item in hw:
        grouped_data[item.dz].append(item) # store the whole item
    
    # create an empty list
    final_nmts_list = []
    
    # Sample from each differential based on dz samples
    for dz, group in grouped_data.items():
        # set the group_size to the size of the group
        group_size = len(group)
        num_samples = max(1, math.ceil(group_size * samepl_size)) # Ensures at least 1 of each differential if it is too small
        
        # Loop through utems in the group
        for item in group[:num_samples]:
            # reassign a new instance of NMTS to nmts_instance that is a sample of the NMTS class
            nmts_instance = NMTS(dx=item.dx, dy=item.dy, dz=item.dz, wt=item.wt)
            final_nmts_list.append(nmts_instance)
    
    # Print the length of the sample
    print(len(final_nmts_list))
    
    
    
    # Return the final instance of the sampled NMTS list
    return final_nmts_list

In [ ]:
def plot_distribution():
    global stratum
    # plot original distribution
    dz_values = [item.dz for item in hw]
    plt.hist(dz_values, bins=50, alpha=0.9, label="Original Distribution", color="#bdc9e1") 
    
    # plot sample distribution
    ##sampled_dz_values = [item.dz for item in stratum]
    sampled_dz_values = [stratum]
    #plt.hist(sampled_dz_values, bins=50, alpha=0.9, label="Samples Distribution", color="#045a8d")  
    
    #Plot histogram and store output
    counts, bins, patches = plt.hist(sampled_dz_values, bins=50, alpha=0.9, label="Samples Distribution", color="#045a8d")
    
    # Get maximum bin count
    max_bin_count = max(counts)
    
    # Plot data
    plt.title("Distribution of output differentials: Original vs Stratified sample")
    plt.xlabel("Output differentials")
    plt.ylabel("Frequency")
    plt.legend(loc="upper right")
    plt.yticks([max_bin_count, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000])
    plt.axhline(y=max_bin_count, color="r", linestyle="--", label=("Y=1000 Line"))
    
    # Generate plot
    plt.show() 
    

In [ ]:
def mask_1(a, b, wshift):
    #initialise variables s1 and var. s1 variable is a flag to control the loop and var calculates the new mask
    s1 = 0
    var = 0
    mask = 2 ** 16 - 1
    
    # Continue the loop as long as s1 equals 0
    while(s1==0):
        
        #shift the cariables a and b to the left by wshift bits value
        # Then perform a bitwise AND operation with the global mask. The bitwise AND is defined by the &
        a1 = (a<<(wshift)) & mask
        b1 = (b<<(wshift)) & mask
        
        # Perform a bitwise OR operation on a1 and b1 variables and store the result in s1
        s1=a1+b1
        
        # If s1 is still equal to zero (indicating no set bits in a1 or b1),
        # increment var variable and decrement wshift by 1 for the next iteration of the loop
        if(s1 == 0):
            var = var + 1
        wshift = wshift - 1
        
        # Calculate mask1 by shifting global mask to the left by var bits
        # then performing bitwise AND with the global mask
        # Ths creates a new mask that zeroes out the var least significant bits
        mask1 = (mask<<var) & mask
        
    # Return the newly calculated mask
    return mask1

In [ ]:
def weightAND(alpha1, beta1, gamma1): 
    # Perform a bitwise AND operation between gamma1 and the bitwise negative of the XOR between aplha1 and beta 1 and assign to the s variable
    # One done, apply the global mask
    s = (gamma1 & (~(alpha1 ^ beta1))) & mask
    
    # Convert the bitwise AND between the XOR of alhpa1 and beta1 and the global mask to a minary string (bin function)
    temp = bin((alpha1 ^ beta1) & mask)
    
    # Calculate the hamming weight as the count of "1's" in the binary string ignoring the first character which is "0b" for the binary representation in Python
    wt = (temp[1:].count("1"))
    
    # If s is not zero return -200 and 200
    # This could be a case where the differential characteristic is not possible or not considered
    if(s != 0):        
        return -200, 200
    else:
        # If s equals zero return the hamming weight and the probability of the differential characteristic
        # The probability is calculated as 2 to the power oof negative weight
        return wt, math.pow(2,-wt)
    

In [ ]:
#right circular shifts
def ROR(x, r):
    # Left shift x by the value of SIMON_TYPE - r places, moving the least significant bits to the most significant bit positions
    # Next, right shift x by r places, moving the r most significant bits to te least significant bit positions
    # Add the 2 value togather to rotate the bits of x a total of r places to the right
    # Apply the mask to ensure the result is within the desired bit size
    x = ((x << (SIMON_TYPE - r)) + (x >> r)) & mask
    
    # Return the binary number of x
    return x

#left circular shifts
def ROL(x, r):
    # Right shift x by the value of SIMON_TYPE - r places, moving the least significant bits to the most significant bit positions
    # Next, left shift x by r places, moving the r most significant bits to te least significant bit positions
    # Add the 2 value togather to rotate the bits of x a total of r places to the left
    # Apply the mask to ensure the result is within the desired bit size
    x = ((x >> (SIMON_TYPE - r)) + (x << r)) & mask
    return x

In [ ]:
# Define a reverse find differential path function
# The fuction holds 4 arguments
# st1 and st0 are the initial states of the input and output
# the SIMON_ROUNDS is the number of rounds
# n counter variable
def find_diff_path_rev(st1, st0, SIMON_ROUNDS, n):
    
    


    # Initialise local variables
    # lp=1              
    temp_wt=0 # Initialise the temporary hamming weight of the differential path
    tempdec_list=[] # Initialise a temporary list to store the differential characteristics
    
    # Loop for the number of defined SIMON_ROUNDS
    for i in range (0, SIMON_ROUNDS):
        # Save initial values as inp1 and inp0
        inp1 = st1
        inp0 = st0
        
        # Swap st1 and st0 for reverse operation
        tempst0 = st0
        st0 = st1
        st1 = tempst0
        
        # Calculate A, B and C values by rotating st1
        A = ROL(st1, alpha)
        B = ROL(st1, beta)
        C = ROL(st1, gamma)
        
        # Initialise hamming weight as -200
        wt = -200
        
        # Loop through the wt variable until a non-negative weight is found
        while(wt == -200):
            # Take a random characteristic from the hw variable
            #op=hw[random.randint(0,len(hw)-1)].dz
            #op = random.choice(stratum).dz
            op = random.choice(stratum)
            
            # Calculate the hamming weight
            wt, wt1 = weightAND(A, B,  op)
            
        # Calculate the value of D from the XOR of op and C
        D=op^C
        
        # Calculate the value of E from the XOR of st0 and D
        E = st0^D
        
        # Uodate st0 for the next round with the value of E
        st0 = E
        
        
        # Add the new charactristic to the list and update the total haming weight
        tempdec_list.append(NMTS(inp1, inp0, op, wt))
        temp_wt = temp_wt + tempdec_list[i].wt
        
    # Increment the counter by 1
    n = n + 1
    
    # Return the list of characteristics, total hamming weight and counter
    return tempdec_list, temp_wt, n

In [ ]:
# Define a find differential path function
# The fuction holds 4 arguments
# st1 and st0 are the initial states of the input and output
# the SIMON_ROUNDS is the number of rounds
# n counter variable
def find_diff_path(st1, st0, SIMON_ROUNDS, n):
    
    
    
    # Initialise local variables
    #lp=1             
    temp_wt = 0 # Initialise the temporary hamming weight of the differential path
    tempdec_list = [] # Initialise a temporary list to store the differential characteristics
    
    # Loop for the number of defined SIMON_ROUNDS
    for i in range (0, SIMON_ROUNDS):
        
        # Calculate A, B and C values by rotating st1
        A=ROL(st1, alpha)
        B=ROL(st1, beta)
        C=ROL(st1, gamma)
        
        # Initialise hamming weight as -200
        wt = -200
        
        # Loop through the wt variable until a non-negative weight is found
        while(wt == -200):
            # Take a random characteristic from the hw variable
            #op = hw[random.randint(0, len(hw)-1)].dz
            #op = random.choice(stratum).dz
            op = random.choice(stratum)
            
            # Calculate the hamming weights
            wt,wt1=weightAND(A,B,op)
        # Calculate the value of D from the XOR of op and C
        D=op^C
        
        # Calculate the value of E from the XOR of st0 and D
        E=st0^D
        
        # Add the new charactristic to the list and update the total haming weight
        tempdec_list.append(NMTS(st1,st0,op,wt)) 
        temp_wt= temp_wt+tempdec_list[i].wt
        
        #update state with new valid output differential
        st0=st1; st1=E
        
    # Increment the counter by 1
    n=n+1
    
    # Return the list of characteristics, total hamming weight and counter
    return tempdec_list, temp_wt, n

In [ ]:
# Define the find_best_path function
# Takes 5 arguments
# st1 and st0 are the initial states of the input and output of the block cypher
# SIMON_ROUNDS is the number of rounds
# wt_above is the cumulative hamming weight from previous iterations
# best_weight is the lowest hamming weight so far
def find_best_path(st1, st0, SIMON_ROUNDS, wt_above, best_wt):
    #using n as index value for list and initialise to 0
    n=0
    
    # Loop through SIMON_ROUNDS beginning at the end and then decrement by one until it reaches the stop value of 0
    for r in range(SIMON_ROUNDS, 0, -1):
        # Find differential path for the round
        tempdec_list, temp_wt, n = find_diff_path(st1, st0, r, n)
        
        # Check if the hamming weight for the path found is less than the best hamming weight so far
        if((temp_wt + wt_above) < best_wt):
            # If the hamming weight is less than the best hamming weight so far update the best hamming weight
            best_wt = temp_wt + wt_above
            
            # Update the dec_list with the new differential path
            for i,j in zip(range(n-1,22), range(len(tempdec_list))):
                dec_list[i] = (NMTS(tempdec_list[j].dx,tempdec_list[j].dy,tempdec_list[j].dz,tempdec_list[j].wt))
                
        # If n is less than SIMON_ROUNDS update st1 and st0 for the next iteration
        if(n < SIMON_ROUNDS):
            # Update to the dx and dy values of the nth dec_list item
            st1 = dec_list[n].dx
            st0 = dec_list[n].dy
            
        # Add the weight of the last character to the cumulative wt_above variable
        wt_above = wt_above + dec_list[n - 1].wt   
    
    # Return the best hamming weight found
    return best_wt

In [ ]:
# Define the find_best_path_rev function
# Takes 5 arguments
# st1 and st0 are the initial states of the input and output of the block cypher
# SIMON_ROUNDS is the number of rounds
# wt_above is the cumulative hamming weight from previous iterations
# best_weight is the lowest hamming weight so far
def find_best_path_rev(st1,st0, SIMON_ROUNDS, wt_above, best_wt):
    #using n as index value for list and initialise it at 0
    n=0
    
    # Loop over rounds in reverse order starting at the number of rounds and decrementing by one until it reaches the stop value of 0
    for r in range(SIMON_ROUNDS,0,-1):
        
        # Find differential path in reverse for the round
        tempdec_list, temp_wt, n = find_diff_path_rev(st1,st0,r, n)
        
        # Check if the hamming weight for the path found is less than the best hamming weight so far
        if((temp_wt+wt_above) < best_wt):
            
            # If the hamming weight is less than the best hamming weight so far update the best hamming weight
            best_wt= temp_wt+wt_above
            
            # Update the dec_list_rev with the new differential path
            for i,j in zip(range(n-1,22), range(len(tempdec_list))):
                dec_list_rev[i]=(NMTS(tempdec_list[j].dx,tempdec_list[j].dy,tempdec_list[j].dz,tempdec_list[j].wt))
        
        # If n is less than SIMON_ROUNDS update st1 and st0 for the next iteration
        if(n < SIMON_ROUNDS):
            # Update to the dx and dy values of the nth dec_list item
            st1=dec_list_rev[n].dx
            st0=dec_list_rev[n].dy
        # Add the weight of the last character to the cumulative wt_above variable
        wt_above= wt_above+dec_list_rev[n-1].wt 
        
    # Return the best hamming weight found
    return best_wt

In [ ]:
class NMTS(object):
    """__init__() functions as the class constructor
    Assign params with default to none if not set
    """
    def __init__(self, dx=None, dy=None, dz=None, wt=None):
        self.dx = dx # Assign the input dx value to the objects dx attribute
        self.dy = dy # Assign the input dy value to the objects dy attribute
        self.dz = dz # Assign the input dz value to the objects dz attribute
        self.wt = wt # Assign the input wt value to the objects wt attribute

In [ ]:
# Define a dictionary to hold row names
row_names = {}

experiment_loops = 1

target_weight = 21 # set to 21

# Loop the code x times to simulate x number of experiments
for j in range(experiment_loops):
    
    # Define variables

    #alpha beta are for left and right circular shift     
    alpha, beta, gamma = 1,8,2
    #SIMON_ROUNDS=12
    SIMON_TYPE=16
    mask = 2 ** SIMON_TYPE - 1
    wshift=15


    bw=999
    s=9999
    hw=[]
    iterations = 0

    # Call the function to populate the global hw variable with a list of differential characteristics        
    highwaylist()
    
    # call the sample data function to get a list of the NMTS objects
    nmts_objects = sample_data(hw)
    
    # Extract sample dz values from the NMTS objects to use as the quota stratum. We will sample from this list to find the path
    stratum = [obj.dz for obj in nmts_objects]
    
    # start the time recording
    start_time = time.time()

    # Loop over each item in hw except for the first and last items
    for ch in range(1,len(nmts_objects)-1):

        # Initialise lists to store the best differential paths found both forward and reverse
        dec_list=[0]*22
        dec_list_rev=[0]*22

        # Initialise the cumulative hamming weight both forward and reverse and set to 0
        wt_above=0
        wt_above_rev=0

        # Initialise the best weight with a large value for both forward and reverse paths
        best_wt=500
        best_wt_rev=500

        # Initialise count to 0
        count=0  

        # Loop until the total hamming weight of forward and reverse is less than 21 or until 20 attempts have been made
        while(bw > target_weight):
        #while(bw > target_weight and iterations < 30256):
        
            iterations = iterations + 1
            
            # Retrieve string values for st1 and st0 from current nmts_objects object. This replaces original hw list
            st1=nmts_objects[ch].dx
            st0=nmts_objects[ch].dy

            # Set he number of SIMONROUNDS to 5
            SIMONROUNDS=5

            # Find the best differential path in forward direction for the current hw and assign to best_wt
            best_wt=find_best_path(st1,st0,SIMONROUNDS,wt_above,best_wt)
            #best_wt=find_best_path(st1,st0,SIMONROUNDS,wt_above,best_wt, find_diff_path)

            # Increment the count
            count=count+1

            # If the count exceeds 20 terminate the loop
            if(count>20):
                break

            # Set SIMONROUNDS_rev to 6
            SIMONROUNDS_rev=6

            # Retrieve string values for st1 and st0 from current nmts_objects object. This replaces original hw list
            st1=nmts_objects[ch].dx
            st0=nmts_objects[ch].dy
            
            ch=ch+1

            # Find the best differential path in forward direction for the current hw and assign to best_wt_rev
            best_wt_rev=find_best_path_rev(st1,st0,SIMONROUNDS_rev,wt_above,best_wt_rev)
            #best_wt=find_best_path(st1,st0,SIMONROUNDS_rev,wt_above,best_wt, find_diff_path_rev)

            # Calculate the total hamming weight of the best forward and reverse paths
            bw=best_wt+best_wt_rev

            # If the total weight is less than the smallest weight found so far update the smallest and print the forward and reverse paths
            if(bw<s):
                s=bw

                # Print the smallest weight so far and the differential characteristic of the ch index
                print(s,ch)

                # Print marker to indicate the direction of the loop
                print("Decryption list is in reverse:")   

                # Loop through SIMONROUNDS_rev from its value to 0 decrementing by 1 each loop
                for i in range(SIMONROUNDS_rev-1,-1,-1): 
                    """
                    Print statement that prints the details of each round
                    Convert to hexadecimal and then displayt he following
                        * Input difference
                        * Output difference
                        * Intermediate difference
                        * The hamming weight for the current round
                    """
                    print("Starting input of %i round and weight:" %i,hex(dec_list_rev[i].dx),hex(dec_list_rev[i].dy),hex(dec_list_rev[i].dz),(dec_list_rev[i].wt))

                # Print marker to indicate the direction of the loop
                print("Decryption list is forward:")   

                # Loop through SIMONROUNDS from 0 to it's value by 1 each loop
                for i in range(0, SIMONROUNDS):   
                    """
                    Print statement that prints the details of each round
                    Convert to hexadecimal and then displayt he following
                        * Left inout
                        * Right input
                        * Output differential
                        * The hamming weight for the current round
                    """
                    print("Starting input of %i round and weight:" %i,hex(dec_list[i].dx), hex(dec_list[i].dy),hex(dec_list[i].dz),(dec_list[i].wt))

                # Display the best weight recorded
                print("Best weight is:", bw)
                print("Iterations:", iterations)
                print("Duration:", time.time() - start_time)

                
                
    end_time = time.time()

    elapsed_time = end_time - start_time

    print("Elapsed time: ", elapsed_time)

    
    df.loc[len(df.index)] = [bw, elapsed_time, iterations]
    
    # Set the current row name
    row_names[j] = "Experiment " + str(j + 1)
    
    # check if it is the last iteration and plot the distribution
    if j == experiment_loops - 1:
        plot_distribution()
    

# rename row names
df = df.rename(index = row_names)   


# add the mean, median, min and max to the table
df.loc["MEAN"] = [df["Best weight"].mean(), df["Duration"].mean(), df["Iterations"].mean()]
df.loc["MEDIAN"] = [df["Best weight"].median(), df["Duration"].median(), df["Iterations"].median()]
df.loc["MIN"] = [df["Best weight"].min(), df["Duration"].min(), df["Iterations"].min()]
df.loc["MAX"] = [df["Best weight"].max(), df["Duration"].max(), df["Iterations"].max()]

# save the csv file 
df.to_csv(r'./simon32_forward_backwards_stratified_1.csv', encoding='utf-8', header='true')

df                